# Импорт библиотек

In [ ]:
# !pip install lightgbm --install-option=--gpu
# !pip install catboost
# !pip install xgboost
# !pip install optuna

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import os
# import optuna
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
# import lightgbm as lgbm
# from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import catboost
from catboost import CatBoostClassifier, Pool
# from optuna.integration import LightGBMPruningCallback, XGBoostPruningCallback
import warnings
warnings.filterwarnings("ignore")

# Чтение данных

In [ ]:
train = pd.read_csv('data/train.csv')
train.head()

In [ ]:
players_feats = pd.read_csv('data/players_feats.csv')
players_feats.head()

In [ ]:
#Фичи команды 1
team1_feats = train.merge(players_feats, how='left', left_on=['team1_id', 'map_id'], right_on=['team_id', 'map_id'])\
    .drop(['p1_id', 'p2_id', 'p3_id', 'p4_id', 'p5_id', 'team_id', 'map_name_y'], axis=1)
team1_feats.head()

In [ ]:
#Фичи команды 2
team2_feats = train.merge(players_feats, how='left', left_on=['team2_id', 'map_id'], right_on=['team_id', 'map_id'])\
    .drop(['p1_id', 'p2_id', 'p3_id', 'p4_id', 'p5_id', 'team_id', 'map_name_y'], axis=1)
team2_feats.head()

In [ ]:
#Дельта фичей
teams_feats = team1_feats.iloc[:,5:-1] - team2_feats.iloc[:,5:-1]
teams_feats

In [ ]:
#Получаем датафрем команда-карта
train_feats = pd.concat([train, teams_feats], axis=1)
train_feats.head()

In [ ]:
#Проверяем пустые значения
missing_data = train_feats.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print(" ")

In [ ]:
train_feats = train_feats.fillna(0)

In [ ]:
#Сохраняем получившийся датасет
train_feats.to_csv('train_feats.csv', index=False)

Подготавливаем тестовый датафрейм

In [ ]:
test = pd.read_csv('data/test.csv').drop(['index'], axis=1)

In [ ]:
#Тестовые фичи команды 1
team1_test_feats = test.merge(players_feats, how='left', left_on=['team1_id', 'map_id'], right_on=['team_id', 'map_id'])\
    .drop(['p1_id', 'p2_id', 'p3_id', 'p4_id', 'p5_id', 'team_id', 'map_name_y'], axis=1)
team1_test_feats

In [ ]:
#Тестовые фичи команды 2
team2_test_feats = test.merge(players_feats, how='left', left_on=['team2_id', 'map_id'], right_on=['team_id', 'map_id'])\
    .drop(['p1_id', 'p2_id', 'p3_id', 'p4_id', 'p5_id', 'team_id', 'map_name_y'], axis=1)
team2_test_feats

In [ ]:
#Дельта тестовых фичей
teams_test_feats = team1_test_feats.iloc[:,5:-1] - team2_test_feats.iloc[:,5:-1]
teams_test_feats

In [ ]:
#Тестовый датасет команда-карта
test_feats = pd.concat([test, teams_test_feats], axis=1)
test_feats

In [ ]:
#Проверяем пустые значения
missing_data = test_feats.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print(" ")

In [ ]:
#Сохраняем получившийся датасет
test_feats.to_csv('test_feats.csv', index=False)

# LGBM + XGB + CatBoost + Optuna

In [2]:
train = pd.read_csv('data/train_feats.csv')
test = pd.read_csv('data/test_feats.csv')#.drop()

In [152]:
train.head()

,map_id,team1_id,team2_id,map_name,who_win,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,...,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
0,289,6665,7718,Ancient,0,132,-10.0,176,-0.20,-0.6,...,0.31,0.46,0.19,135,89,34,13,0.62,0.12,7.4
1,715,4411,10577,Inferno,0,-345,-13.7,-317,-0.14,-3.1,...,-0.23,-0.51,-0.17,-11,-113,-17,24,-1.60,-0.38,-0.5
2,157,11251,9455,Nuke,1,293,13.0,182,0.31,25.0,...,-0.08,0.23,0.09,49,40,-21,-14,-0.39,-0.36,28.7
3,524,4608,7532,Mirage,0,108,-24.5,-55,0.54,1.5,...,0.25,0.28,0.21,152,79,59,36,0.59,0.36,7.8
4,404,8637,6667,Overpass,1,33,5.8,59,-0.09,-9.1,...,-0.03,-0.07,0.01,-44,-15,0,7,-0.82,0.08,0.0


In [3]:
drop_cols_train = [
    'who_win',
    'map_name',
]

In [4]:
train_x = train.drop(columns=drop_cols_train, axis=1)
train_y = train[['who_win']]

In [155]:
def Optuna(argument):
    N_TRIALS = 20
    N_SPLITS = 5

    def LGBMObjective(trial, train_x = train_x, train_y = train_y):
        param = {
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 8, 4096),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000),
            "lambda_l1": trial.suggest_int("lambda_l1", 0, 100),
            "lambda_l2": trial.suggest_int("lambda_l2", 0, 100),
            "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.95),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.95),
            "device_type": 'gpu',
            "n_estimators": 10000,
            "bagging_freq": 1,
            "metric": 'auc',
            "objective": 'binary'
        }
        param = {
            "device_type": trial.suggest_categorical("device_type", ['gpu']),
            "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
            "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
            "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
            "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
            "bagging_fraction": trial.suggest_float(
              "bagging_fraction", 0.2, 0.95, step=0.1
            ),
            "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
            "feature_fraction": trial.suggest_float(
              "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }
        cv = StratifiedKFold(n_splits=5, shuffle=True)
        cv_scores = np.empty(5)
        for idx, (train_idx, test_idx) in enumerate(cv.split(train_x, train_y)):
            X_train, X_valid = train_x.iloc[train_idx], train_x.iloc[test_idx]
            y_train, y_valid = train_y.iloc[train_idx], train_y.iloc[test_idx]
            model = LGBMClassifier(**param)
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric="roc_auc_score", early_stopping_rounds=100,
                callbacks=[LightGBMPruningCallback(trial, "auc")], verbose = False
            )
            preds = model.predict_proba(X_valid)
            cv_scores[idx] = roc_auc_score(y_valid, preds[:,1])
        return np.mean(cv_scores)

    def XGBObjective(trial, x_train = train_x, y_train = train_y): 
        param = {
            'max_depth': trial.suggest_int('max_depth', 6, 10), 
            'n_estimators': trial.suggest_int('n_estimators', 400, 4000, 400), 
            'eta': trial.suggest_float('eta', 0.007, 0.013), 
            'subsample': trial.suggest_discrete_uniform('subsample', 0.2, 0.9, 0.1),
            'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.2, 0.9, 0.1),
            'colsample_bylevel': trial.suggest_discrete_uniform('colsample_bylevel', 0.2, 0.9, 0.1),
            'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 1e4), 
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 1e4),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 1e4), 
            'gamma': trial.suggest_loguniform('gamma', 1e-4, 1e4),
            'predictor': "gpu_predictor",
            'eval_metric' : 'auc',
            'objective' : 'binary:logistic',
            'tree_method': 'gpu_hist',
        }
        cv = StratifiedKFold(n_splits=5, shuffle=True)
        cv_scores = np.empty(5)
        for idx, (train_idx, test_idx) in enumerate(cv.split(train_x, train_y)):
            X_train, X_valid = train_x.iloc[train_idx], train_x.iloc[test_idx]
            y_train, y_valid = train_y.iloc[train_idx], train_y.iloc[test_idx]
            model = XGBClassifier(**param)
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100, 
                      callbacks=[XGBoostPruningCallback(trial, 'validation_0-auc')], verbose = 0)
            preds = model.predict_proba(X_valid)
            cv_scores[idx] = roc_auc_score(y_valid, preds[:,1])
        return np.mean(cv_scores)

    def CatBoostObjective(trial, x_train = train_x, y_train = train_y):
        param = {
            'iterations':trial.suggest_int("iterations", 1000, 20000),
            'objective': trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
            'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
            'od_wait':trial.suggest_int('od_wait', 500, 2000),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.02,1),
            'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
            'random_strength': trial.suggest_uniform('random_strength',10,50),
            'depth': trial.suggest_int('depth',1,15),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
            'verbose': False,
            'task_type' : 'GPU',
            'devices' : '0',
            'eval_metric':'AUC',
            'od_type': 'IncToDec',
            'od_pval': 1e-7, 
            'od_wait' : 100,
        }
        if param['bootstrap_type'] == 'Bayesian':
            param['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
        elif param['bootstrap_type'] == 'Bernoulli':
            param['subsample'] = trial.suggest_float('subsample', 0.1, 1)
        cv = StratifiedKFold(n_splits=5, shuffle=True)
        cv_scores = np.empty(5)
        for idx, (train_idx, test_idx) in enumerate(cv.split(train_x, train_y)):
            X_train, X_valid = train_x.iloc[train_idx], train_x.iloc[test_idx]
            y_train, y_valid = train_y.iloc[train_idx], train_y.iloc[test_idx]
            model = CatBoostClassifier(**param)
            #Notice: Optuna Callback not supported on CatBoost
            cat_features = list(range(0, 4))
            model.fit(X_train, y_train, cat_features=list(range(0, 4)), eval_set=[(X_valid, y_valid)])
            preds = model.predict_proba(X_valid)
            cv_scores[idx] = roc_auc_score(y_valid, preds[:,1])
        return np.mean(cv_scores)
    def ObjectiveSelector(argument):
        objective = {
            'LGBM': LGBMObjective,
            'XGB': XGBObjective,
            'CatBoost': CatBoostObjective            
        }
        return objective.get(argument, "Invalid Selection")
    def ModelSelector(argument, trial): #Switch case not usable here without crashing. 
        if(argument == 'LGBM'):
            return LGBMClassifier(**trial.params)
        elif(argument == 'XGB'):
            return XGBClassifier(**trial.params)
        elif(argument == 'CatBoost'):
            return CatBoostClassifier(**trial.params)
        return "Invalid Model"
    study = optuna.create_study(direction="maximize")
    study.optimize(ObjectiveSelector(argument), n_trials=N_TRIALS)
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    model = ModelSelector(argument, trial)
    model.fit(train_x, train_y)
    predictions_optuna = model.predict_proba(test)
    return predictions_optuna

In [ ]:
argument = 'XGB'
predictions_optuna = Optuna(argument)



In [ ]:
argument = 'LGBM'
predictions_optuna = Optuna(argument)

In [ ]:
argument = 'CatBoost'
predictions_optuna = Optuna(argument)

In [133]:
predictions_optuna

array([[0.06760734, 0.93239266],
       [0.8856727 , 0.11432731],
       [0.29068255, 0.70931745],
       [0.48067194, 0.51932806],
       [0.1266132 , 0.8733868 ],
       [0.41669893, 0.58330107],
       [0.39425468, 0.6057453 ],
       [0.27800858, 0.7219914 ],
       [0.2008478 , 0.7991522 ],
       [0.3807056 , 0.6192944 ],
       [0.75064164, 0.24935834],
       [0.22646284, 0.77353716],
       [0.8717536 , 0.12824643],
       [0.6314043 , 0.36859572],
       [0.19562536, 0.80437464],
       [0.67187494, 0.32812506],
       [0.83487487, 0.16512513],
       [0.3517853 , 0.6482147 ],
       [0.8120623 , 0.18793766],
       [0.8825827 , 0.11741725],
       [0.8470104 , 0.15298961],
       [0.5521674 , 0.44783255],
       [0.11282182, 0.8871782 ],
       [0.43720257, 0.5627974 ],
       [0.5889373 , 0.41106272],
       [0.555501  , 0.444499  ],
       [0.55359936, 0.44640067],
       [0.24048668, 0.7595133 ],
       [0.6778593 , 0.3221407 ],
       [0.7440591 , 0.25594088],
       [0.

# H2O AUTOML

In [56]:
!apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


In [57]:
!pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.5 MB 41 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.3-py2.py3-none-any.whl size=177541401 sha256=74886f836dbb583319296d4e29201924f4711e7b692d0d97c3986aa779b505ee
  Stored in directory: /root/.cache/pip/wheels/16/f8/f4/69e1ff6a0d1cb61bdbc0d9888ee8437a1acf1eb1c6ffb8be20
Successfully built h2o


In [58]:
import h2o

In [59]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp907e9v3_
  JVM stdout: /tmp/tmp907e9v3_/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp907e9v3_/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_hrfwna
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [60]:
from h2o.automl import H2OAutoML

In [61]:
train_df = h2o.H2OFrame(train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [62]:
train_df

map_id,team1_id,team2_id,map_name,who_win,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,p1_assists_per_round,p1_deaths_per_round,p1_saved_by_teammate_per_round,p1_saved_teammates_per_round,p1_rating,p1_kill_death,p1_kill_round,p1_rounds_with_kills,p1_kill_death_difference,p1_total_opening_kills,p1_total_opening_deaths,p1_opening_kill_ratio,p1_opening_kill_rating,p1_team_win_percent_after_first_kill,p1_first_kill_in_won_rounds,p2_total_kills,p2_headshots,p2_total_deaths,p2_kd_ratio,p2_damage_per_round,p2_grenade_damage_per_round,p2_maps_played,p2_rounds_played,p2_kills_per_round,p2_assists_per_round,p2_deaths_per_round,p2_saved_by_teammate_per_round,p2_saved_teammates_per_round,p2_rating,p2_kill_death,p2_kill_round,p2_rounds_with_kills,p2_kill_death_difference,p2_total_opening_kills,p2_total_opening_deaths,p2_opening_kill_ratio,p2_opening_kill_rating,p2_team_win_percent_after_first_kill,p2_first_kill_in_won_rounds,p3_total_kills,p3_headshots,p3_total_deaths,p3_kd_ratio,p3_damage_per_round,p3_grenade_damage_per_round,p3_maps_played,p3_rounds_played,p3_kills_per_round,p3_assists_per_round,p3_deaths_per_round,p3_saved_by_teammate_per_round,p3_saved_teammates_per_round,p3_rating,p3_kill_death,p3_kill_round,p3_rounds_with_kills,p3_kill_death_difference,p3_total_opening_kills,p3_total_opening_deaths,p3_opening_kill_ratio,p3_opening_kill_rating,p3_team_win_percent_after_first_kill,p3_first_kill_in_won_rounds,p4_total_kills,p4_headshots,p4_total_deaths,p4_kd_ratio,p4_damage_per_round,p4_grenade_damage_per_round,p4_maps_played,p4_rounds_played,p4_kills_per_round,p4_assists_per_round,p4_deaths_per_round,p4_saved_by_teammate_per_round,p4_saved_teammates_per_round,p4_rating,p4_kill_death,p4_kill_round,p4_rounds_with_kills,p4_kill_death_difference,p4_total_opening_kills,p4_total_opening_deaths,p4_opening_kill_ratio,p4_opening_kill_rating,p4_team_win_percent_after_first_kill,p4_first_kill_in_won_rounds,p5_total_kills,p5_headshots,p5_total_deaths,p5_kd_ratio,p5_damage_per_round,p5_grenade_damage_per_round,p5_maps_played,p5_rounds_played,p5_kills_per_round,p5_assists_per_round,p5_deaths_per_round,p5_saved_by_teammate_per_round,p5_saved_teammates_per_round,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
289,6665,7718,Ancient,0,132,-10,176,-0.2,-0.6,1.5,10,271,-0.12,0.07,0.01,0.08,-0.03,-0.04,-0.2,-0.12,102,-44,16,18,0.06,-0.03,15.7,-1.2,130,-12.7,167,-0.14,-16.2,-1,10,271,-0.15,0.01,-0.06,0,0.02,-0.13,-0.14,-0.15,93,-37,11,24,-0.24,-0.3,10.9,-11.3,255,4.3,173,0.39,22.1,1,9,248,0.28,-0.01,-0.01,0.05,-0.02,0.37,0.39,0.28,152,82,61,51,0.56,0.48,6.4,16.1,31,2.8,21,0.04,5.5,2.1,1,27,0.06,0.03,0.03,0.07,0.06,0.01,0.04,0.06,26,10,-6,5,-1.57,-0.23,11.1,-4.8,217,-10.7,128,0.46,15.8,3.1,9,248,0.19,-0.02,-0.13,-0.1,0.07,0.31,0.46,0.19,135,89,34,13,0.62,0.12,7.4
715,4411,10577,Inferno,0,-345,-13.7,-317,-0.14,-3.1,3.7,-20,-516,-0.07,0.05,0.02,-0.01,-0.01,-0.06,-0.07,-0.03,-111,-3,-27,-46,0.72,0.03,-7,-1.2,-193,-4.5,-246,0.08,6,-1.4,-15,-393,0.07,0.02,0.02,0,0.06,0.03,0.12,0.08,-29,50,-20,-38,0.22,0,-4.2,-1.5,-309,-21.9,-221,-0.19,-2.6,6.1,-15,-393,-0.06,0.04,0.07,-0.01,0,-0.14,-0.18,-0.06,-83,-64,9,28,-0.34,0.09,-11.5,5.6,-422,10.1,-278,-0.28,-7.5,-3.4,-19,-493,-0.12,-0.03,0.04,0.03,-0.04,-0.13,-0.24,-0.13,-149,-88,-15,5,-0.45,0.11,13.7,8.4,-190,14.8,-49,-0.43,-3.6,2.8,-7,-180,-0.13,0.05,0.11,0.03,0.04,-0.23,-0.51,-0.17,-11,-113,-17,24,-1.6,-0.38,-0.5
157,11251,9455,Nuke,1,293,13,182,0.31,25,1.3,9,259,0.23,0.02,0.03,0.03,0.03,0.35,0.21,0.16,92,45,60,19,0.97,0.57,7.8,17.9,201,-4.2,145,0.11,3,-1.1,9,259,0.05,-0.01,-0.01,-0.04,-0.03,0.05,0.13,0.05,89,33,14,5,0.32,0.02,14.9,0.7,99,12.7,168,-0.32,-19.7,-0.4,9,259,-0.21,-0.04,0,-0.01,-0.05,-0.26,-0.3,-0.22,48,-53,-2,9,-0.6,-0.24,-6.4,-9.5,221,2

In [63]:
train_df.describe()

Rows:713
Cols:124

,map_id,team1_id,team2_id,map_name,who_win,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,p1_assists_per_round,p1_deaths_per_round,p1_saved_by_teammate_per_round,p1_saved_teammates_per_round,p1_rating,p1_kill_death,p1_kill_round,p1_rounds_with_kills,p1_kill_death_difference,p1_total_opening_kills,p1_total_opening_deaths,p1_opening_kill_ratio,p1_opening_kill_rating,p1_team_win_percent_after_first_kill,p1_first_kill_in_won_rounds,p2_total_kills,p2_headshots,p2_total_deaths,p2_kd_ratio,p2_damage_per_round,p2_grenade_damage_per_round,p2_maps_played,p2_rounds_played,p2_kills_per_round,p2_assists_per_round,p2_deaths_per_round,p2_saved_by_teammate_per_round,p2_saved_teammates_per_round,p2_rating,p2_kill_death,p2_kill_round,p2_rounds_with_kills,p2_kill_death_difference,p2_total_opening_kills,p2_total_opening_deaths,p2_opening_kill_ratio,p2_opening_kill_rating,p2_team_win_percent_after_first_kill,p2_first_kill_in_won_rounds,p3_total_kills,p3_headshots,p3_total_deaths,p3_kd_ratio,p3_damage_per_round,p3_grenade_damage_per_round,p3_maps_played,p3_rounds_played,p3_kills_per_round,p3_assists_per_round,p3_deaths_per_round,p3_saved_by_teammate_per_round,p3_saved_teammates_per_round,p3_rating,p3_kill_death,p3_kill_round,p3_rounds_with_kills,p3_kill_death_difference,p3_total_opening_kills,p3_total_opening_deaths,p3_opening_kill_ratio,p3_opening_kill_rating,p3_team_win_percent_after_first_kill,p3_first_kill_in_won_rounds,p4_total_kills,p4_headshots,p4_total_deaths,p4_kd_ratio,p4_damage_per_round,p4_grenade_damage_per_round,p4_maps_played,p4_rounds_played,p4_kills_per_round,p4_assists_per_round,p4_deaths_per_round,p4_saved_by_teammate_per_round,p4_saved_teammates_per_round,p4_rating,p4_kill_death,p4_kill_round,p4_rounds_with_kills,p4_kill_death_difference,p4_total_opening_kills,p4_total_opening_deaths,p4_opening_kill_ratio,p4_opening_kill_rating,p4_team_win_percent_after_first_kill,p4_first_kill_in_won_rounds,p5_total_kills,p5_headshots,p5_total_deaths,p5_kd_ratio,p5_damage_per_round,p5_grenade_damage_per_round,p5_maps_played,p5_rounds_played,p5_kills_per_round,p5_assists_per_round,p5_deaths_per_round,p5_saved_by_teammate_per_round,p5_saved_teammates_per_round,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill
type,int,int,int,enum,int,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real,real,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real,real,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real,real,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real,real,int,real,int,real,real,real,int,int,real,real,real,real,real,real,real,real,int,int,int,int,real,real,real
mins,0.0,4411.0,4411.0,,0.0,-730.0,-51.6,-711.0,-2.78,-91.1,-13.3,-41.0,-1097.0,-0.97,-0.19,-0.74,-0.12,-0.14,-1.41,-2.8,-0.98,-372.0,-238.0,-108.0,-82.0,-19.14,-1.33,-100.0,-37.3,-698.0,-57.9,-660.0,-1.62,-84.5,-15.4,-36.0,-975.0,-0.77,-0.2199999999999999,-0.7,-0.16,-0.16,-1.17,-1.62,-0.85,-338.0,-150.0,-103.0,-106.0,-7.91,-1.36,-87.5,-25.5,-648.0,-60.5,-642.0,-1.64,-90.0,-9.6,-35.0,-947.0,-0.82,-0.15,-0.65,-0.11,-0.15,-1.33,-3.0,-0.9,-310.0,-140.0,-90.0,-93.0,-4.43,-1.32,-90.9,-23.8,-684.0,-55.0,-649.0,-1.72,-81.2,-13.8,-34.0,-921.0,-0.79,-0.17,-0.73,-0.17,-0.14,-1.31,-1.75,-1.05,-325.0,-167.0,-102.0,-114.0,-5.83,-1.83,-100.0,-31.2,-675.0,-54.1,-576.0,-1.17,-80.4,-16.7,-35.0,-913.0,-0.74,-0.15,-0.68,-0.13,-0.18,-1.15,-1.97,-0.95,-296.0,-156.0,-85.0,-76.0,-3.52,-1.5,-100.0
mean,370.92566619915823,7243.643758765781,7787.844319775595,,0.4586255259467041,9.214586255259468,-0.6809256661991582,12.0224403927068

In [64]:
train_df['who_win'] = train_df['who_win'].asfactor()
train_df['map_id'] = train_df['map_id'].asfactor()
train_df['team1_id'] = train_df['team1_id'].asfactor()
train_df['team2_id'] = train_df['team2_id'].asfactor()
train_df['map_name'] = train_df['map_name'].asfactor()

In [111]:
feats_train,feats_test,feats_valid = train_df.split_frame(ratios=[.7, .15], seed = 1234)

In [108]:
y = "who_win"
x = train_df.columns
x.remove(y)
# x.remove('map_id')
# x.remove('team1_id')
# x.remove('team2_id')
x.remove('map_name')

In [119]:
aml = H2OAutoML(max_models = 20, seed = 10, exclude_algos = ["StackedEnsemble","DeepLearning"], nfolds=0)

In [120]:
aml.train(x = x, y = y, training_frame = feats_train, validation_frame=feats_valid)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_18_20221205_53117


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    30                 30                          7171                   1            3            2             2             4             3.16667

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.002114816042690116
RMSE: 0.04598712909815219
LogLoss: 0.04579653186590943
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9061292748928211
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      273  0    0        (0.0/273.0)
1      0    240  0        (0.0/240.0)
Total  273  240  0        (0.0/513.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value    idx
---------------------------  -----------  -------  -----
max f1                       0.906129     1        139
max f2                       0.906129     1        139
max f0point5                 0.906129     1        139
max accuracy                 0.906129     1        139
max precision                0.966817     1        0
max recall                   0.906129     1        139
max specificity              0.966817     1        0
max absolute_mcc             0.906129     1        139
max min_per_class_accuracy   0.906129     1        139
max mean_per_class_accuracy  0.906129     1        139
max tns                      0.966817     273      0
max fns                      0.966817     219      0
max fps                      0.03052      273      372
max tps                      0.906129     240      139
max tnr                      0.966817     1        0
max fnr                      0.966817     0.9125   0
max fpr                      0.03052      1        372
max tpr                      0.906129     1        139

Gains/Lift Table: Avg response rate: 46.78 %, avg score: 46.87 %
group    cumulative_data_fraction    lower_threshold    lift    cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0409357                   0.966817           2.1375  2.1375             1                0.966817   1                           0.966817            0.0875          0.0875                     113.75   113.75             0.0875
2        0.0409357                   0.966804           0       2.1375             0                0          1                           0.966817            0               0.0875                     -100     113.75             0.0875
3        0.0721248                   0.96679            2.1375  2.1375             1                0.96679    1                           0.966805            0.0666667       0.154167                   113.75   113.75             0.154167
4        0.105263                    0.961912           2.1375  2.1375             1                0.9635     1                           0.965765            0.0708333       0.225                      113.75   113.75             0.225
5        0.150097                    0.95908            2.1375  2.1375             1                0.960871   1                           0.964303            0.0958333       0.320833                   113.75   113.75             0.320833
6        0.20078                   

# BEST_MODELS PREDICTIONS

In [122]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_x, train_y, test_size=0.33, random_state=42)

In [134]:
best_catboost_model = CatBoostClassifier(
    random_seed=63,
    iterations=10484, 
    objective='Logloss', 
    bootstrap_type='Bernoulli', 
    od_wait=1253, 
    learning_rate=0.5769703684226246, 
    reg_lambda=49.71966280773916, 
    random_strength=40.96205179962597,
    depth=4,
    min_data_in_leaf=22,
    leaf_estimation_iterations=11,
    subsample=0.7018775670299524
)
best_catboost_model.fit(
    train_x, train_y,
    cat_features = list(range(0,3)),
    verbose=False,
    eval_set=(X_validation, y_validation),
)

In [141]:
test.shape

(30, 122)

In [148]:
X_test = test.drop(columns='map_name', axis=1)
test_pool = Pool(data=X_test, cat_features = list(range(0,3)))
catboost_predictions = best_catboost_model.predict_proba(test_pool)
print('Predictoins:')
print(catboost_predictions)

Predictoins:
[[0.7628852  0.2371148 ]
 [0.03179758 0.96820242]
 [0.0601858  0.9398142 ]
 [0.62865761 0.37134239]
 [0.67417876 0.32582124]
 [0.28897144 0.71102856]
 [0.01842492 0.98157508]
 [0.98080634 0.01919366]
 [0.98429574 0.01570426]
 [0.95665964 0.04334036]
 [0.99762475 0.00237525]
 [0.0135302  0.9864698 ]
 [0.13428326 0.86571674]
 [0.57837446 0.42162554]
 [0.97058939 0.02941061]
 [0.30379931 0.69620069]
 [0.03659122 0.96340878]
 [0.64636171 0.35363829]
 [0.42012679 0.57987321]
 [0.7082391  0.2917609 ]
 [0.05917491 0.94082509]
 [0.77401044 0.22598956]
 [0.6083034  0.3916966 ]
 [0.84360106 0.15639894]
 [0.97220863 0.02779137]
 [0.88679725 0.11320275]
 [0.89622877 0.10377123]
 [0.05354486 0.94645514]
 [0.99305455 0.00694545]
 [0.3150578  0.6849422 ]]


In [150]:
f = open('submit.csv', 'w')
f.write('Id,Action\n')
for idx in range(len(catboost_predictions)):
    line = str(test['map_id'][idx]) + ',' + str(catboost_predictions[idx][1]) + '\n'
    f.write(line)
f.close()